<a href="https://colab.research.google.com/github/HecMartinSol/03MAIR-Alg.Optimizacion/blob/master/AG3/hector_martin_solis_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3 - Actividad guiada

Héctor Martín Solís

https://colab.research.google.com/drive/1l1VtlepYeVYw2AoOCKWIcpETvrXTFnGJ

In [3]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f5a36c65ba8>)

In [6]:
!pip install tsplib95

In [0]:
import tsplib95
import random
import math

In [0]:
problem = tsplib95.load_problem(file)

In [0]:
Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())

In [0]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion

In [21]:
crear_solucion(Nodos)

[0,
 31,
 20,
 35,
 17,
 38,
 6,
 3,
 30,
 2,
 29,
 33,
 40,
 25,
 10,
 23,
 8,
 41,
 12,
 4,
 24,
 11,
 21,
 22,
 1,
 5,
 16,
 37,
 15,
 28,
 27,
 32,
 39,
 18,
 13,
 36,
 14,
 26,
 9,
 7,
 34,
 19]

In [0]:
#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)

In [25]:
distancia(1, 2, problem)

34

In [0]:
#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [0]:
def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion(Nodos)
  mejor_valor = distancia_total(mejor_solucion, problem)
  for i in range(N):
    solucion = crear_solucion(Nodos)
    valor = distancia_total(solucion, problem)
    if valor < mejor_valor:
      mejor_solucion = solucion
      mejor_valor = valor
    
  print("la mejor solucion es", mejor_valor, mejor_solucion)


In [38]:
busqueda_aleatoria(problem,1000)

la mejor solucion es 3567 [0, 41, 10, 19, 27, 3, 38, 12, 23, 9, 32, 39, 8, 25, 18, 4, 16, 37, 26, 28, 6, 5, 2, 11, 22, 40, 24, 21, 36, 17, 1, 15, 14, 7, 29, 31, 33, 35, 20, 13, 30, 34]


In [54]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion
      
solucion = crear_solucion(Nodos)
mejor_solucion = genera_vecina(solucion)

print(mejor_solucion)

[0, 29, 16, 1, 15, 18, 11, 6, 33, 13, 5, 23, 7, 3, 4, 32, 28, 25, 9, 19, 37, 10, 34, 20, 26, 8, 14, 27, 40, 39, 38, 24, 12, 30, 35, 36, 31, 2, 17, 41, 22, 21]


In [0]:
#####################################################################
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina
  
  print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)

In [62]:
busqueda_local(problem, 500)

La mejor solución encontrada es  1667 [0, 20, 33, 34, 32, 4, 6, 26, 10, 25, 41, 40, 24, 21, 39, 9, 23, 8, 22, 38, 30, 29, 28, 2, 27, 3, 1, 31, 35, 36, 17, 5, 18, 12, 11, 13, 19, 14, 16, 15, 37, 7]


In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

In [66]:
solucion = crear_solucion(Nodos)
mejor_solucion = genera_vecina(solucion)
print(solucion)
print(mejor_solucion)

[0, 19, 7, 17, 9, 3, 34, 6, 30, 5, 33, 8, 27, 36, 15, 37, 31, 28, 12, 10, 39, 13, 20, 38, 29, 11, 25, 41, 40, 35, 14, 1, 26, 24, 2, 4, 16, 23, 18, 21, 22, 32]
[0, 19, 7, 17, 9, 3, 34, 6, 30, 5, 33, 8, 27, 36, 15, 37, 31, 28, 12, 10, 39, 13, 20, 38, 29, 11, 25, 41, 40, 35, 14, 1, 26, 18, 2, 4, 16, 23, 24, 21, 22, 32]


In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False


def bajar_temperatura(T):
  return T*.9999

In [73]:
#####################################################################
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100

  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=100000)

	****Encontrada mejor solucion: 4530 [0, 24, 20, 40, 41, 6, 2, 27, 17, 7, 5, 37, 25, 19, 13, 18, 39, 32, 30, 16, 4, 12, 35, 21, 10, 14, 36, 15, 23, 11, 28, 26, 3, 29, 22, 33, 1, 9, 38, 34, 31, 8] 

	****Encontrada mejor solucion: 4524 [0, 22, 29, 8, 3, 6, 26, 27, 14, 7, 5, 37, 25, 4, 40, 21, 2, 32, 15, 16, 38, 12, 28, 34, 30, 18, 36, 11, 23, 13, 39, 41, 10, 20, 24, 33, 1, 17, 35, 9, 31, 19] 

	****Encontrada mejor solucion: 4463 [0, 5, 29, 8, 3, 18, 40, 33, 10, 39, 22, 19, 25, 16, 4, 21, 34, 32, 15, 28, 38, 31, 35, 2, 30, 27, 7, 11, 23, 13, 37, 36, 14, 20, 12, 6, 1, 17, 41, 9, 24, 26] 

	****Encontrada mejor solucion: 4397 [0, 30, 5, 37, 21, 39, 9, 16, 22, 4, 29, 31, 36, 19, 1, 32, 10, 17, 20, 24, 41, 13, 15, 7, 33, 26, 14, 2, 27, 11, 25, 23, 6, 12, 8, 40, 3, 18, 34, 28, 35, 38] 

	****Encontrada mejor solucion: 4352 [0, 30, 5, 37, 21, 39, 9, 16, 22, 8, 29, 31, 36, 19, 1, 32, 10, 17, 20, 24, 41, 13, 15, 7, 33, 26, 14, 2, 27, 11, 25, 23, 6, 12, 4, 40, 3, 18, 35, 28, 34, 38] 

	****Encon

In [0]:
def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [0]:
# Colonia de hormigas
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia = distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)

In [90]:
hormigas(problem, 1000)

[0, 1, 29, 22, 8, 39, 28, 2, 6, 32, 31, 26, 30, 33, 20, 13, 38, 4, 41, 10, 12, 9, 5, 19, 24, 25, 23, 34, 7, 27, 17, 15, 14, 11, 21, 40, 3, 16, 36, 35, 37, 18]
3791
